<a href="https://colab.research.google.com/github/orgoca/W207FinalProject/blob/main/Project2EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
# Code to read csv file from Google Drive into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [144]:
#Import Libraries
import os
import pandas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
import csv

In [145]:
#Load test DF
testLink = 'https://drive.google.com/file/d/1chKVOcXwNzGRe_hRxpGlq18NVaSRM8X9/view?usp=sharing'
idTest = testLink.split('/')[-2]
downloaded = drive.CreateFile({'id':idTest}) 
downloaded.GetContentFile('test.csv')  
test = pd.read_csv('test.csv')

In [146]:
#Load train DF
trainLink = 'https://drive.google.com/file/d/1fP-AWiUGLjLkL0c_u67HOUFmHOMO7qm2/view?usp=sharing'
idTrain = trainLink.split('/')[-2]
downloaded = drive.CreateFile({'id':idTrain}) 
downloaded.GetContentFile('train.csv')  
train = pd.read_csv('train.csv')

In [147]:
#Split Training Data into MiniTrain and MiniTest (Test data has no labels so impossible to measure error from it)
miniTrain, miniTest = train_test_split(train, test_size=0.2)

In [148]:
#Create a vectorizer, create question and target lists, and vectorize train questions
vectorizer = CountVectorizer()
trainQuestions = miniTrain.question_text.tolist()
trainLabels = miniTrain.target.tolist()
vTrain = vectorizer.fit_transform(trainQuestions)

In [127]:
#What is the vocabulary size, and create a bag of words
print("vocab size: ", vTrain.shape)
bagOfWords = vectorizer.vocabulary_
len(bagOfWords)

vocab size:  (1044897, 172948)


172948

In [152]:
#Create a logistic regression model and fit to train data
lr_clf = LogisticRegression(C=1)
lr_clf.fit(vTrain, trainLabels)

train_predict = lr_clf.predict(vTrain)
train_predict

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0, 0, 0, ..., 0, 0, 0])

In [153]:
lr_clf.predict_proba(vTrain)

array([[0.99280442, 0.00719558],
       [0.9956997 , 0.0043003 ],
       [0.99461185, 0.00538815],
       ...,
       [0.96670189, 0.03329811],
       [0.99870052, 0.00129948],
       [0.7517752 , 0.2482248 ]])

In [176]:
#Try an insincere question. Insincere questions should get a higher probability score to legitimate questions.
testQuestionInsincere = "Why do Europeans say they're the superior race, when in fact it took them over 2,000 years until mid 19th century to surpass China's largest economy?"
vdev = vectorizer.transform([testQuestionInsincere.lower()])

lr_clf.predict(vdev)

lr_clf.predict_proba(vdev)

array([[0.38301993, 0.61698007]])

In [178]:
#Try a sincere question. Sincere questions should get a lower probability score ideally close to zero.
testQuestionInsincere = "How many ounces are there in a gallon?"
vdev = vectorizer.transform([testQuestionInsincere.lower()])

lr_clf.predict(vdev)

lr_clf.predict_proba(vdev)

array([[0.98983737, 0.01016263]])